In [1]:
#https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/input_data.py

"""Functions for downloading and reading MNIST data."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import tempfile

import numpy
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

/Users/r3dmaohong/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
mnist = read_data_sets('MNIST_data/', one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 100

In [4]:
n_input = 784
n_classes = 10

In [5]:
dropout = 0.75

In [6]:
x = tf.placeholder(tf.float32, [None, n_input])
_X = tf.reshape(x, shape=[-1,28,28,1])

In [7]:
y = tf.placeholder(tf.float32, [None, n_classes])

In [8]:
def conv2d(img, w, b):
    return(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1,1,1,1], padding='SAME'),b)))

In [9]:
# first layer
wc1 = tf.Variable(tf.random_normal([5,5,1,32])) #
bc1 = tf.Variable(tf.random_normal([32])) # 32 feature maps?

conv1 = conv2d(_X, wc1, bc1)

In [10]:
def max_pool(img, k):
    return(tf.nn.max_pool(img, ksize = [1,k,k,1], strides=[1,k,k,1], padding='SAME'))

In [11]:
conv1 = max_pool(conv1, k=2)

In [12]:
keep_prob = tf.placeholder(tf.float32)
conv1 = tf.nn.dropout(conv1, keep_prob)

In [13]:
# second layer
wc2 = tf.Variable(tf.random_normal([5,5,32,64])) #
bc2 = tf.Variable(tf.random_normal([64]))

conv2 = conv2d(conv1, wc2, bc2)

In [14]:
conv2 = max_pool(conv2, k=2)

In [15]:
conv2 = tf.nn.dropout(conv2, keep_prob)

In [16]:
# fully connected layer?
wd1 = tf.Variable(tf.random_normal([7*7*64, 1024]))
bd1 = tf.Variable(tf.random_normal([1024]))

In [17]:
dense1 = tf.reshape(conv2, [-1, wd1.get_shape().as_list()[0]])

In [18]:
dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, wd1), bd1))

In [19]:
dense1 = tf.nn.dropout(dense1, keep_prob)

In [20]:
# output
wout = tf.Variable(tf.random_normal([1024, n_classes]))
bout = tf.Variable(tf.random_normal([n_classes]))

In [21]:
pred = tf.add(tf.matmul(dense1, wout), bout)

In [22]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

In [23]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [24]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [25]:
init = tf.global_variables_initializer()

In [26]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step*batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size=batch_size)
        sess.run(optimizer, feed_dict={x:batch_xs, y:batch_ys, keep_prob:dropout})
        
        if step % display_step==0:
            acc = sess.run(accuracy, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.})
            loss = sess.run(cost, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 12800, Minibatch Loss= 2439.287842, Training Accuracy= 0.79688
Iter 25600, Minibatch Loss= 1768.154175, Training Accuracy= 0.85938
Iter 38400, Minibatch Loss= 1213.325928, Training Accuracy= 0.90625
Iter 51200, Minibatch Loss= 645.977417, Training Accuracy= 0.91406
Iter 64000, Minibatch Loss= 1157.865234, Training Accuracy= 0.91406
Iter 76800, Minibatch Loss= 1079.931396, Training Accuracy= 0.89844
Iter 89600, Minibatch Loss= 302.322174, Training Accuracy= 0.92969
Optimization Finished!
Testing Accuracy: 0.953125
